In [0]:
from google.colab import auth, drive

auth.authenticate_user()

In [8]:
!earthengine authenticate

Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

/usr/local/lib/python3.6/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
Traceback (most recent call last):
  File "/usr/local/bin/earthengine", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/dist-packages/ee/cli/eecli.py", line 92, in main
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/platform/app.py", line 40, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 299, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 250, in _run_main
    sys.exit(main(argv))
  File "/usr/local/lib/python3.6/dist-packages/ee/cli/eecli.py", line 81, in _run_command
    dispatc

KeyboardInterrupt: ignored

In [20]:
# Import the Earth Engine API and initialize it.
import ee
import time
import os
import pandas as pd
from pprint import pprint
import numpy as np

import tensorflow as tf
from keras.optimizers import adam
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout

# Montamos la unidad de trabajo de nuestro drive
drive.mount('/content/drive/')

# Cambiamos la ruta de trabajo a nuestro drive
os.chdir('/content/drive/My Drive/')

ee.Initialize()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Funciones a utilizar

In [0]:
# Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(bands).divide(10000)

def normalizedDifference(a, b):
  """Compute normalized difference of two inputs.

  Compute (a - b) / (a + b).  If the denomenator is zero, add a small delta.  

  Args:
    a: an input tensor with shape=[1]
    b: an input tensor with shape=[1]

  Returns:
    The normalized difference as a tensor.
  """
  nd = (a - b) / (a + b)
  nd_inf = (a - b) / (a + b + 0.000001)
  return tf.where(tf.math.is_finite(nd), nd, nd_inf)

Ejemplo 1.1 - Preprocesar datos de Earth Engine y obtener las tablas en CSV.

In [0]:
# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
label = 'landcover'

# Use Landsat 8 surface reflectance data.
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
# The image input data is a 2018 cloud-masked median composite.
image = l8sr.filterDate('2018-01-01', '2018-12-31').map(maskL8sr).median()

# Change the following two lines to use your own training data.
labels = ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')

# Sample the image at the points and add a random column.
sample = image.sampleRegions(collection=labels, properties=[label], scale=30).randomColumn()

# Partition the sample approximately 70-30.
training = sample.filter(ee.Filter.lt('random', 0.7))
testing = sample.filter(ee.Filter.gte('random', 0.7))

# Names for output files.
trainFilePrefix = 'Training_demo'
testFilePrefix = 'Testing_demo'
fileNameSuffix = '.csv'

# This is list of all the properties we want to export.
featureNames = list(bands)
featureNames.append(label)

trainingTask = ee.batch.Export.table.toDrive(
    collection=training,
    folder='agricultura',
    description='Training Export', 
    fileNamePrefix=trainFilePrefix, 
    fileFormat='CSV',
    selectors=featureNames)

testingTask = ee.batch.Export.table.toDrive(
    collection=testing,
    folder='agricultura',
    description='Testing Export', 
    fileNamePrefix=testFilePrefix, 
    fileFormat='CSV',
    selectors=featureNames)

# Start the tasks.
trainingTask.start()
testingTask.start()

# Poll the training task until it's done.
while trainingTask.active():
  print('Polling for task (id: {}).'.format(trainingTask.id))
  time.sleep(30)
print('Done with training export.')

# Poll the testing task until it's done.
while testingTask.active():
  print('Polling for task (id: {}).'.format(testingTask.id))
  time.sleep(30)
print('Done with testing export.')

Ejemplo 1.2 - Obtener el fichero csv y prepararlo para keras

In [0]:
# Get the tfRecord from Google drive
trainFilePath = 'agricultura/bands tables/' + trainFilePrefix + fileNameSuffix
testFilePath = 'agricultura/bands tables/' + testFilePrefix + fileNameSuffix

# Get the dataframes
trainDataFrame = pd.read_csv(trainFilePath)
print('csv for training loaded')
testDataFrame = pd.read_csv(testFilePath)
print('csv for testing loaded')

x_train = trainDataFrame.iloc[:,:-1]
y_train = trainDataFrame.iloc[:,-1]

x_test = testDataFrame.iloc[:,:-1]
y_test = testDataFrame.iloc[:,-1]

# Number of features
num_features_input = x_train.shape[1]
num_classes = y_train.iloc[-1,]+1

# convert class vectors to binary class matrices, and convert the inputs in a numpy array
x_train = x_train.to_numpy()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = y_test.to_numpy()
y_test = keras.utils.to_categorical(y_test, num_classes)

Ejemplo 1.3 - Crear el modelo y empezar a entrenar

In [0]:
# hyperparameters
lr = 1e-03
batch_size = 128
epochs = 10
shuffle = True

# Create the model
model = Sequential()
model.add(Dense(1024, activation='relu', kernel_initializer='random_normal', input_dim=num_features_input))
model.add(Dense(512, activation='relu', kernel_initializer='random_normal'))
model.add(Dense(num_classes, activation='softmax', kernel_initializer='random_normal'))

# Compiling the neural network
model.compile(
    optimizer=adam(lr=lr), 
    loss='categorical_crossentropy', 
    metrics =['accuracy'])

# Training the model
model.fit(
    x=x_train, 
    y=y_train, 
    batch_size=batch_size, 
    epochs=epochs, 
    shuffle=shuffle,
    verbose=1)

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])